# ATP Tennis Data - Symmetric Raw Data


In our feature engineering notebooks, we created a dataset for match where we left categorical columns un-encoded. Player 1 was randomly chosen to be either a winner or loser for the match so that we can have multiple classes to train our models.

We created an alternate dataset, where samples were duplicated and split into 2 samples where each row would have player 1 as a winner and 2nd row would have player 1 as the loser for the same match.

Hypothesis for this notebook is that there shouldn't be any significant difference in our accuracy as duplicating the data adds no new information for our models.

We are going to run this dataset through our models to see what effects this has on model accuracy.

NOTE: Again, we are using raw player stats and we have not encoded our categorical columns

## Results from Gradient Boosting with Raw Player Data
```
Model Score: 0.6826468602295747

ROC/AUC Score: 0.682614201415636
              precision    recall  f1-score   support

        Loss       0.68      0.67      0.68      7381
         Win       0.68      0.69      0.69      7429

    accuracy                           0.68     14810
   macro avg       0.68      0.68      0.68     14810
weighted avg       0.68      0.68      0.68     14810
```


# Summary of Results

* All models except for KNN did slightly worse (although probably not significant). We have slightly higher recall on losses but we traded off the same amount of recall for win recalls
* Our best accuracy score came from Gradient Boosting again
* Because the difference in accuracy score is small, it is inconclusive whether using data from this method of feature engineering would be effective

```
Model Score: 0.6800472653612424

ROC/AUC Score: 0.680048494074675
              precision    recall  f1-score   support

        Loss       0.68      0.68      0.68     14815
         Win       0.68      0.68      0.68     14805

    accuracy                           0.68     29620
   macro avg       0.68      0.68      0.68     29620
weighted avg       0.68      0.68      0.68     29620
```






In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from datetime import datetime
import pickle
import json
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from util import jupyter_util as ju
from util.model_util import ModelWrapper, RSTATE, N_JOBS, MAX_ITER, REPORT_FILE, LABEL_COL

%matplotlib inline
sns.set()


# date
DATE_FORMAT = '%Y-%m-%d'
DATE = datetime.now().strftime(DATE_FORMAT)


DESCRIPTION = "raw-sym"

# sometimes I run these notebooks via command line. Environment variable is set so we know whether we are in DEBUG mode or not
# if you want to manually run this in DEBUG mode, change default value to True
DEBUG = bool(os.environ.get("IPYNB_DEBUG", False))

if DEBUG:
    FEATURE_FILE = f'../datasets/atp_matches_1985-2019_featuressym_test.csv'
    ModelWrapper.REPORT_FILE = '../reports/summary-test.csv'
    DESCRIPTION = f"{DESCRIPTION}-test"
else:

    FEATURE_FILE = f'../datasets/atp_matches_1985-2019_featuressym.csv'


START_YEAR = 1998
END_YEAR = 2018



In [2]:
X_train, X_test, y_train, y_test = ju.get_data(FEATURE_FILE, LABEL_COL, START_YEAR, END_YEAR)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Logistic Regression

In [ ]:

# train our model

lr = ModelWrapper(LogisticRegression(n_jobs = N_JOBS, verbose = 1, random_state = RSTATE),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_lr = lr.predict()

In [ ]:
lr.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_lr)

In [ ]:
lr.save()

# KNN

In [ ]:

# train our model

knn = ModelWrapper(KNeighborsClassifier(n_jobs = N_JOBS),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_knn = knn.predict()

In [ ]:
knn.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_knn)

In [ ]:
knn.save()

# Decision Tree

In [ ]:

# train our model

dt = ModelWrapper(DecisionTreeClassifier(random_state = RSTATE),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_dt = dt.predict()

In [ ]:
dt.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_dt)

In [ ]:
dt.save()

# Random Forest Classifier

In [ ]:

# train our model

rf = ModelWrapper(RandomForestClassifier(random_state=RSTATE, verbose=1),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_rf = rf.predict()

In [ ]:
rf.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_rf)

In [ ]:
rf.save()

# Gradient Boosting

In [ ]:

# train our model

gb = ModelWrapper(GradientBoostingClassifier(random_state=RSTATE, verbose=1, n_iter_no_change = 4),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_gb = gb.predict()

In [ ]:
gb.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_gb)

In [ ]:
gb.save()

# AdaBoost

In [ ]:
ab = ModelWrapper(AdaBoostClassifier(random_state=RSTATE),
                  description = DESCRIPTION, 
                 data_file = FEATURE_FILE,
                  start_year = START_YEAR,
                  end_year = END_YEAR,
                   X_train = X_train,
                   y_train = y_train,
                   X_test = X_test,
                   y_test = y_test).fit()
y_predict_ab = ab.predict()

In [ ]:
ab.analyze()

In [ ]:
# ju.plot_2d(X_test, y_predict_ab)

In [ ]:
ab.save()

# Summary

Decision Tree Classifier did the best with a 92% accuracy score while KNN did the worst at about 60% (below our null hypothesis)

In [ ]:
report = pd.read_csv(ab.report_file)
report.tail(6)

In [ ]:
f, a = plt.subplots(1, 1, figsize=(20, 10))

descs = report.description.unique()

for desc in descs:
    report_desc = report[report.description == desc]
    if desc == DESCRIPTION:
        sns.lineplot(x="model_name", y="accuracy", data=report_desc, label=desc, linewidth=4)
    else:
        sns.lineplot(x="model_name", y="accuracy", data=report_desc, label=desc, linewidth=0.75)

a.axhline(0.65, ls='--', color='r')
a.set_title(f"Model Accuracy")
_ = a.legend(loc="lower left")

In [ ]:
current_report = report[(report.model_name == 'GradientBoostingClassifier') &
                                  (report.description == DESCRIPTION)]
mw = ModelWrapper.get_model_wrapper_from_report(current_report)


In [ ]:
import shap

shap.initjs()

explainer = shap.TreeExplainer(mw.model, data=X_train.values)
shap_values = explainer.shap_values(X_test.values)

In [ ]:
shap.summary_plot(shap_values, X_test, max_display=30)

In [ ]:
print(f'Last Finished: {datetime.now().strftime("%Y-%m-%d %H:%M")}')